<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/vllm_prefix_cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vllm flash-attention triton peft

In [1]:
from vllm import SamplingParams, LLM

texts = ["God this guy is such a douchebag",
  "I love bunnies, don't you?",
  "Hey you dickhead, stop being so stupid",
  "You're a complete idiot, how did you even get this job?",
  "I hope you have a wonderful day!",
  "This product is absolute garbage, don't waste your money",
  "Your opinion is worthless, just like you",
  "Thank you for your help, I really appreciate it",
  "Why don't you go jump off a cliff?",
  "I respectfully disagree with your point of view",
  "You're nothing but a useless piece of trash",
  "Let's work together to find a solution",
  "I wish you would just disappear forever"]
sampling_params = SamplingParams(temperature=0.01, top_p=0.95, max_tokens=5)

In [2]:
def formatting_func(example):
    input = f"""
    <|user|>
    Examine the text and determine  whether the text is toxic or not. Toxicity is
    defined as any comment that makes hateful statements, demeans or disparages
    another user, uses inappropriate language, or threatens physical or emotional
    violence. Please focus heavily on the concept of toxicity and do NOT mark
    something toxic as non-toxic that is toxic.

    Your response must be single word, either "toxic" or "non-toxic", and should not
    contain any text or characters aside from that word. "toxic" means that the text
    meets the definition of toxic. "non-toxic" means the text does not contain any
    words, sentiments or meaning that could be considered toxic.

    You are examining written text content. Here is the text:
        [BEGIN DATA]
        ************
        [Text]: {example}
        ************
        [END DATA]

    You must only reply with a single word: either "toxic", or "non-toxic".
    <|end|>
    <|assistant|>
    """
    return input

In [3]:
prompts = []
for text in texts:
  input = formatting_func(text)
  prompts.append(input)

In [4]:
from vllm.lora.request import LoRARequest

llm = LLM(model="microsoft/Phi-3-mini-4k-instruct", enable_lora=True, gpu_memory_utilization=0.6)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 09-19 12:46:55 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='microsoft/Phi-3-mini-4k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-mini-4k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/Phi-3-mini-4k-instruct, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_cac

/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 09-19 12:46:57 model_runner.py:997] Starting to load model microsoft/Phi-3-mini-4k-instruct...
INFO 09-19 12:46:57 selector.py:240] Cannot use FlashAttention-2 backend due to sliding window.
INFO 09-19 12:46:57 selector.py:116] Using XFormers backend.
INFO 09-19 12:46:58 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-19 12:47:01 model_runner.py:1008] Loading model weights took 7.1227 GB
INFO 09-19 12:47:04 gpu_executor.py:122] # GPU blocks: 966, # CPU blocks: 682
INFO 09-19 12:47:06 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-19 12:47:06 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-19 12:47:41 model_runner.py:1430] Graph capturing finished in 35 secs.


In [4]:
from huggingface_hub import snapshot_download

toxic_lora_path = snapshot_download(repo_id="grounded-ai/phi3-toxicity-judge")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [6]:
import time
start_time = time.time()
outputs = llm.generate(
    prompts,
    sampling_params,
    lora_request=LoRARequest("toxicity_adapter", 1, toxic_lora_path)
)
print(f"Time taken without prefix: {time.time() - start_time}")

<ipython-input-6-4e96696b2a47>:6: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora_request=LoRARequest("toxicity_adapter", 1, toxic_lora_path)
Processed prompts: 100%|██████████| 13/13 [00:00<00:00, 15.14it/s, est. speed input: 3973.36 toks/s, output: 57.06 toks/s]

Time taken without prefix: 0.9476668834686279


In [7]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: '\n    <|user|>\n    Examine the text and determine  whether the text is toxic or not. Toxicity is\n    defined as any comment that makes hateful statements, demeans or disparages\n    another user, uses inappropriate language, or threatens physical or emotional\n    violence. Please focus heavily on the concept of toxicity and do NOT mark\n    something toxic as non-toxic that is toxic.\n\n    Your response must be single word, either "toxic" or "non-toxic", and should not\n    contain any text or characters aside from that word. "toxic" means that the text\n    meets the definition of toxic. "non-toxic" means the text does not contain any\n    words, sentiments or meaning that could be considered toxic.\n\n    You are examining written text content. Here is the text:\n        [BEGIN DATA]\n        ************\n        [Text]: God this guy is such a douchebag\n        ************\n        [END DATA]\n\n    You must only reply with a single word: either "toxic", or "non-toxic

In [5]:
from vllm.lora.request import LoRARequest
llm_prefix_caching = LLM(model="microsoft/Phi-3-mini-4k-instruct", enable_lora=True, gpu_memory_utilization=0.6, enable_prefix_caching=True, disable_sliding_window=True)
outputs = llm_prefix_caching.generate(prompts[0],
    sampling_params,
    lora_request=LoRARequest("toxicity_adapter", 1, toxic_lora_path))

INFO 09-19 12:52:46 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='microsoft/Phi-3-mini-4k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-mini-4k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2047, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/Phi-3-mini-4k-instruct, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_cac

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-19 12:52:51 model_runner.py:1008] Loading model weights took 7.1227 GB
INFO 09-19 12:52:54 gpu_executor.py:122] # GPU blocks: 970, # CPU blocks: 682
INFO 09-19 12:52:57 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-19 12:52:57 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-19 12:53:30 model_runner.py:1430] Graph capturing finished in 34 secs.
INFO 09-19 12:53:30 block_manager_v1.py:263] Automatic prefix caching is enabled.


<ipython-input-5-1605770fa229>:5: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora_request=LoRARequest("toxicity_adapter", 1, toxic_lora_path))
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.23it/s, est. speed input: 852.07 toks/s, output: 9.72 toks/s]


In [6]:
import time
start_time = time.time()
outputs = llm_prefix_caching.generate(
    prompts,
    sampling_params,
    lora_request=LoRARequest("toxicity_adapter", 1, toxic_lora_path)
)
print(f"Time taken with prefix cache: {time.time() - start_time}")

<ipython-input-6-90f5340793a5>:6: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora_request=LoRARequest("toxicity_adapter", 1, toxic_lora_path)
Processed prompts: 100%|██████████| 13/13 [00:00<00:00, 24.43it/s, est. speed input: 6413.61 toks/s, output: 92.10 toks/s]

Time taken with prefix cache: 0.54833984375


In [7]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: '\n    <|user|>\n    Examine the text and determine  whether the text is toxic or not. Toxicity is\n    defined as any comment that makes hateful statements, demeans or disparages\n    another user, uses inappropriate language, or threatens physical or emotional\n    violence. Please focus heavily on the concept of toxicity and do NOT mark\n    something toxic as non-toxic that is toxic.\n\n    Your response must be single word, either "toxic" or "non-toxic", and should not\n    contain any text or characters aside from that word. "toxic" means that the text\n    meets the definition of toxic. "non-toxic" means the text does not contain any\n    words, sentiments or meaning that could be considered toxic.\n\n    You are examining written text content. Here is the text:\n        [BEGIN DATA]\n        ************\n        [Text]: God this guy is such a douchebag\n        ************\n        [END DATA]\n\n    You must only reply with a single word: either "toxic", or "non-toxic